In [2]:
from numpy import exp
from math import sqrt
import numpy as np
from numpy import random
from math import pi

import matplotlib.pyplot as plt
from scipy.stats import norm
from scipy.stats import multivariate_normal

In [3]:
# Densite du Noise
def pn(x, m0, s0):
    return multivariate_normal.pdf(x, m0, s0);



## densité de la loi a posteriori non normalisée:
def pm0(x, m , s ):
    k= len(m)
    return multivariate_normal.pdf(x, m, s)*np.sqrt(np.linalg.det(s)*(2*pi)**k);


In [4]:
s = [[2.0, 0.3], [0.3, 0.5]]
m = [1,2]
x = [[1,2], [3,4]]
pn(x,m,s)
pm0(x,m,s)

array([1.        , 0.01536253])

In [16]:
multivariate_normal.rvs(m,s,3)

array([[0.27843976, 2.96049733],
       [1.00003533, 1.09523898],
       [2.41533996, 3.31728591]])

In [5]:
class Gradient:


    # instance attribute
    def __init__(self, cte,mu,sigma, error_mu,error_sigma, error_cte, ctes,mus,sigmas):
        self.cte = cte
        self.mu = mu
        self.sigma = sigma
        
        self.error_mu = error_mu
        self.error_sigma = error_sigma
        self.error_cte = error_cte 
        
        self.ctes = ctes
        self.mus = mus
        self.sigmas = sigmas
        

## NCE for a d-D distribution

Data $\begin{pmatrix} X_{1} \\ X_{2}\\...\\ X_{d} \end{pmatrix} \sim  N(m,\Sigma)$

Noise $Y \sim  Q = \mu +\sigma N(0,1)$ with $\mu, \sigma$ fixed (in the code it is fixed at mu_unit and sigma_init)

In [6]:
def NCEDescent(x_batches, m, s,mu_init =10, sigma_init=10, cte_init = 0.2, learning_rate = [0.01,0.01], max_iters = 500):    
    
    m0 = mu_init 
    s0 =sigma_init
    cte = cte_init
    
    #not used for nce
    error_mu = [] 
    error_sigma = []
    error_cte = [] 
    
    mus = []
    sigmas = []
    ctes = []
    
    batch_size= len(x_batches[0])
    Q= multivariate_normal.rvs(m0,s0,(10,batch_size))
     
    for itr in range(max_iters): 
        
        for i in range(len(x_batches)):
            x= x_batches[i]
            q=Q[i]
            ctes.append(cte)
    
            #Gradient in respect to the constant
            grad_cte = 1/cte - pm0(x,m,s)/(cte*pm0(x,m,s)+pn(x, m0,s0)) -  pm0(q,m,s)/(cte*pm0(q,m,s)+pn(q,m0,s0))
            grad_cte = np.sum(grad_cte)/batch_size
            cte = cte + learning_rate[0] * grad_cte
            error_cte.append( (grad_cte) ) 
            
    result = Gradient(cte,m0,s0, error_mu,error_sigma, error_cte, ctes,mus,sigmas)
    return result


In [24]:
np.sqrt(np.linalg.det(s)*(2*pi)**len(m))

5.993776774285581

In [7]:
# d =2
m = [1,2]
s = [[2.0, 0.5], [0.5, 3]]

batch_size=100
x_batches = multivariate_normal.rvs(m, s, (10,batch_size))


grad=NCEDescent(x_batches,m, s,mu_init = m, sigma_init=s, cte_init = 5, learning_rate = [0.01,0.01], max_iters = 500)    
print("constant estimate",grad.cte)
print("true constant value" , 1/np.sqrt(np.linalg.det(s)*(2*pi)**len(m)))

constant estimate 0.06637219940618727
true constant value 0.06637219940618727


In [8]:
#d =3
m = [1,2,3]
s = [[1.0, 0.5,0.7], [0.5,1, 0.8], [0.7,0.8, 0.9]]

batch_size=100
x_batches = multivariate_normal.rvs(m, s, (10,batch_size))

### very sensitive

grad=NCEDescent(x_batches,m, s,mu_init = m, sigma_init=s, cte_init = 5, learning_rate = [0.0055,0.01], max_iters = 800)    
print("constant estimate",grad.cte)
print("true constant value" , 1/np.sqrt(np.linalg.det(s)*(2*pi)**len(m)))

constant estimate 0.19594561451853698
true constant value 0.19594561451853681


In [9]:
#d =4
m = [1,2,3,4]
s = [[1.0, 0.5, 0.7, 0.14], 
     [0.5, 1 ,  0.8, 0.24],
     [0.7, 0.8, 0.9, 0.34], 
     [0.14, 0.24, 0.34, 0.8] ]

batch_size=100
x_batches = multivariate_normal.rvs(m, s, (10,batch_size))

### versy sensitive

grad=NCEDescent(x_batches,m, s,mu_init = m, sigma_init=s, cte_init = 5, learning_rate = [0.0055,0.01], max_iters = 800)    
print("constant estimate",grad.cte)
print("true constant value" , 1/np.sqrt(np.linalg.det(s)*(2*pi)**len(m)))

constant estimate 0.10076169582167607
true constant value 0.10076169582167606
